<a href="https://colab.research.google.com/github/pcmbs/preset-embedding_audio-model-colab/blob/main/colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
#@title
# download .env file
!gdown "12ctXEf6BFTb-eCLrgecLuWdQVM8_P85G"

Downloading...
From: https://drive.google.com/uc?id=12ctXEf6BFTb-eCLrgecLuWdQVM8_P85G
To: /content/.env
100% 3.81k/3.81k [00:00<00:00, 22.5MB/s]


In [2]:
#@title
# download nsynth-lite
!rm -r sample_data
!mkdir data && cd data && gdown "1720jJyIB1BAVoGg5Caq1SF78Gy4vZuEt" && unzip -q -o nsynth-lite.zip && rm nsynth-lite.zip && rm -r __MACOSX

Downloading...
From: https://drive.google.com/uc?id=1720jJyIB1BAVoGg5Caq1SF78Gy4vZuEt
To: /content/data/nsynth-lite.zip
100% 1.65G/1.65G [00:10<00:00, 150MB/s]


In [3]:
#@title
# clone GitHub repo
!pip install python-dotenv --quiet
import dotenv
import os
dotenv.load_dotenv(".env")
key = os.getenv("RSA_KEY")

!mkdir -p /root/.ssh
with open("/root/.ssh/id_rsa", "w", encoding="utf8") as fh:
  fh.write(key)
!ssh-keyscan github.com >> /root/.ssh/known_hosts
!chmod 600 /root/.ssh/id_rsa
!ssh -T git@github.com
!git clone git@github.com:pcmbs/preset-embedding_audio-model-colab.git

# change current directory
%cd /content/preset-embedding_audio-model-colab

# move .env file in project folder
import shutil
shutil.move("/content/.env", "/content/preset-embedding_audio-model-colab/.env")

# github.com:22 SSH-2.0-babeld-c89ab1f3
# github.com:22 SSH-2.0-babeld-dc5ec9be
# github.com:22 SSH-2.0-babeld-4a94ee6b
# github.com:22 SSH-2.0-babeld-dc5ec9be
# github.com:22 SSH-2.0-babeld-dc5ec9be
Hi pcmbs/preset-embedding_audio-model-colab! You've successfully authenticated, but GitHub does not provide shell access.
Cloning into 'preset-embedding_audio-model-colab'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 265 (delta 92), reused 162 (delta 54), pack-reused 60
Receiving objects: 100% (265/265), 52.82 MiB | 25.40 MiB/s, done.
Resolving deltas: 100% (94/94), done.
/content/preset-embedding_audio-model-colab


'/content/preset-embedding_audio-model-colab/.env'

In [4]:
#@title
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.8/728.8 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 7.8 MB/s eta 0:00:

# Code


Run the following cell to print the current yaml config

In [5]:
#@title

from hydra import initialize, compose
from hydra.core.hydra_config import HydraConfig
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="configs", job_name="print_cfg"):
    cfg = compose(config_name="evaluate_embeddings", return_hydra_config=True)
    cfg.hydra.job.num = 0 # arbitrary value, otherwise raise an error since missing
    cfg.hydra.runtime.output_dir = cfg.hydra.run.dir # same as above, should be that for single run
    HydraConfig.instance().set_config(cfg)
    cfg_no_hydra = {k: v for k, v in cfg.items() if k != "hydra"} # dont print hydra config
    print(OmegaConf.to_yaml(cfg_no_hydra, resolve=True))


seed: 667
similarity: pairwise_cosine_similarity
task_name: encodec_48khz
data:
  root: /content/data/nsynth-lite
  sources:
  - acoustic
  - electric
  - synthetic
  audio_length: 4.0
  batch_size: 32
  num_samples: 3072
  shuffle: true
logger:
  _target_: wandb.init
  dir: /content/preset-embedding_audio-model-colab/logs/encodec_48khz/2023-07-12_22-07-24
  mode: online
  id: null
  anonymous: null
  project: audio_model_selection
  group: encodec_48khz
  tags: []
model:
  encoder:
    _target_: src.models.encodec.encoder.EncodecEncoder.encodec_model_48khz
    repository: /content/preset-embedding_audio-model-colab/checkpoints
    segment: 4.0
    overlap: 0.0
  name: encodec_48khz
metrics:
  relative_pairwise_dists:
    num_samples: 10
    n_neighbors: 5
    mode:
    - nearest
    - linspace
paths:
  root_dir: /content/preset-embedding_audio-model-colab
  data_dir: /content/data/nsynth-lite
  log_dir: /content/preset-embedding_audio-model-colab/logs
  ckpt_dir: /content/preset-embed

In [5]:
!python src/evaluate_embeddings.py

Global seed set to 667
[2023-07-12 22:33:00,381][src.data.nsynth][INFO] - Loading NSynth data from split train at /content/data/nsynth-lite/nsynth-train
[2023-07-12 22:33:00,856][src.data.nsynth][INFO] - 	Found 12850 samples.
[2023-07-12 22:33:00,926][__main__][INFO] - Instantiating model <src.models.encodec.encoder.EncodecEncoder.encodec_model_48khz>
[2023-07-12 22:33:01,516][__main__][INFO] - Instantiating wandb logger
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B account'
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.15.5
wandb

# git push

In [ ]:
!git config --global user.email "paolocombes@gmail.com"
!git config --global user.name "Paolo Combes"
!git add .
!git commit -m "removed torchinfo logging"
!git push origin main